In [1]:
import os
import datetime
import numpy as np
import pandas as pd
import geopandas as gpd

import eflowcalc

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
print(eflowcalc.__version__)

0.1.0


In [3]:
path_to_dataset = "/home/georgy/Documents/PEAK/results/tier2"

In [4]:
basins = gpd.read_file(os.path.join(path_to_dataset, "gis/basins.geojson"))

In [5]:
def mm2cms(mm, area):
    # convert cms to mm
    coef = (86400*1000)/(area*1000000)
    return mm / coef

In [6]:
def calculate_eflow_characteristics(basin_id, mode="HST", gcm="MIROC5", rcp="rcp26"):
        
    if mode == "HST":
        
        # define years to consider
        years = [str(i) for i in range(1979,2017)]
        
        path_runoff = os.path.join(path_to_dataset, "hydro/runoff/", mode, f"{basin_id}.csv")
        
    elif mode == "PRJ":
        
        # define years to consider
        years = [str(i) for i in range(2017,2100)]
        
        path_runoff = os.path.join(path_to_dataset, "hydro/runoff/", mode, gcm, rcp, f"{basin_id}.csv")
        
    # read input data
    runoff = pd.read_csv(path_runoff, 
                         index_col=0, 
                         parse_dates=True)
    
    basin_area = float(basins[basins["idx"]==str(basin_id)]["area_merit"].iloc[0])
    
    # convert from mm to cms
    runoff["cms"] = mm2cms(runoff["Runoff"], basin_area)
    
    holder = {}
    
    for year in years[1:]:
        
        year = int(year)
        
        characteristics = eflowcalc.calculator(sfcs=eflowcalc.everything,
                                               datetimes=runoff[f"{year-1}-11-01":f"{year}-10-31"].index.to_pydatetime(),
                                               streamflows=runoff[f"{year-1}-11-01":f"{year}-10-31"]["cms"].to_numpy(),
                                               drainage_area=basin_area,
                                               hydro_year="01/11")
        
        characteristics = np.where(np.isfinite(characteristics.ravel())!=True, np.nan, characteristics.ravel())
        
        holder[year] = characteristics.tolist()
    
    df_chars = pd.DataFrame(holder, index=[i.__name__ for i in eflowcalc.everything]).T
    
    return df_chars

In [7]:
%%time
nvkz = calculate_eflow_characteristics(10240, mode="HST")

CPU times: user 11.4 s, sys: 3.6 ms, total: 11.4 s
Wall time: 11.4 s


In [8]:
%%time
nvkzf = calculate_eflow_characteristics(10240, mode="PRJ")

CPU times: user 24.4 s, sys: 7.89 ms, total: 24.4 s
Wall time: 24.4 s
